In [1]:
from mp_api.client.mprester import MPRester
from matminer import featurizers
from pymatgen.core import Structure, Composition
from pymatgen.core.periodic_table import Element
import pandas as pd
import numpy as np

In [2]:
API_KEY = "fqPPo7Czb5mkbFh8mltlZd0I33csuKv0"
mpr = MPRester(API_KEY)

In [3]:
df = pd.read_csv("data.csv")
mp_id_request = df["mp_id"].to_list()


In [4]:
docs=mpr.summary.search(material_ids=mp_id_request, all_fields=True)

Retrieving SummaryDoc documents:   0%|          | 0/398 [00:00<?, ?it/s]

In [64]:
df_data = pd.DataFrame(columns=[ 'mp_id', 'formula', 'n_sites', 'crystal system', 'band_gap', 'formation_energy_per_atom'])
for i in docs:
    data=[i.material_id.string, i.formula_pretty, i.nsites,  i.symmetry.symbol, i.band_gap, i.formation_energy_per_atom]
    df_data.loc[len(df_data)]=data


In [65]:
df_data

,mp_id,formula,n_sites,crystal system,band_gap,formation_energy_per_atom
0,mp-1174572,Li7Mn2(CoO4)3,24,C2/m,0.0000,-1.746340
1,mp-1174053,Li3MnCoO5,20,C2/m,1.6399,-1.958115
2,mp-754136,CsLi7(NiO3)2,16,C2/m,2.4418,-1.661172
3,mp-1173894,Li2CoO3,12,C2/m,0.0000,-1.686714
4,mp-1100487,Li9Mn7O16,32,C2/m,1.2325,-2.126954
...,...,...,...,...,...,...
393,mp-1100753,Li9Mn2Co5O16,32,C2/m,0.0000,-1.722323
394,mp-1174060,Li5Mn(CoO4)2,16,C2/m,0.0000,-1.803946
395,mp-758018,Li3P2WO8,14,C2/m,4.1832,-2.545702
396,mp-1181529,KLi6BiO6,14,C2/m,1.2849,-1.714463


In [25]:
from matminer.featurizers import conversions, composition, structure

In [68]:
help(composition.ElementProperty)

Help on class ElementProperty in module matminer.featurizers.composition.composite:

class ElementProperty(matminer.featurizers.base.BaseFeaturizer)
 |  ElementProperty(data_source, features, stats)
 |  
 |  Class to calculate elemental property attributes.
 |  
 |  To initialize quickly, use the from_preset() method.
 |  
 |  Features: Based on the statistics of the data_source chosen, computed
 |  by element stoichiometry. The format generally is:
 |  
 |  "{data source} {statistic} {property}"
 |  
 |  For example:
 |  
 |  "PymetgenData range X"  # Range of electronegativity from Pymatgen data
 |  
 |  For a list of all statistics, see the PropertyStats documentation; for a
 |  list of all attributes available for a given data_source, see the
 |  documentation for the data sources (e.g., PymatgenData, MagpieData,
 |  MatscholarElementData, etc.).
 |  
 |  Args:
 |      data_source (AbstractData or str): source from which to retrieve
 |          element property data (or use str for

In [69]:
ele_fraction_featurizer = composition.ElementFraction()
meredig_featuizer = composition.Meredig()
fractionlist = [] 
meredig_list = []
for i in df_data["formula"]:
    composite = Composition(i)
    ele_fraction = ele_fraction_featurizer.featurize(composite)
    meredig = meredig_featuizer.featurize(composite)
    fractionlist.append(ele_fraction)
    meredig_list.append(meredig)
df_data["element fraction"] = fractionlist
df_data["meredig"] = meredig_list

In [72]:
df_data.to_csv("data_feature.csv")


In [76]:
data_formation = df_data[["mp_id", "formation_energy_per_atom"]]
data_formation.to_csv("data_formation_energy.csv", index=False, header=False)